In [21]:
import torch
from src.dataset.modelnet40 import load_modelnet40
from torch.utils.data import DataLoader, Dataset
import sys
import os

from tqdm import tqdm

In [22]:
train_ds, test_ds = load_modelnet40("data/ModelNet40_2")
test_dl = DataLoader(test_ds, batch_size=32)

xbox (40/40): 100%|██████████| 20/20 [00:00<00:00, 317750.30it/s]


In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(true_labels, predicted_labels, average='weighted', zero_division=0)
    f1 = f1_score(true_labels, predicted_labels, average='weighted', zero_division=0)

    return accuracy, precision, recall, f1

def compute_test_metrics(model, test_dl):
    model.eval()
    
    trues = []
    preds = []

    with torch.no_grad():
        for batch in tqdm(test_dl):
            x = batch["pointcloud"].float().to('mps')
            y = batch["category"].to("mps")

            # Assuming model returns probabilities, you might need to convert them to classes.
            pred_probabilities = model(x)
            pred_classes = pred_probabilities.argmax(dim=1)

            trues.extend(y.cpu().numpy())
            preds.extend(pred_classes.cpu().numpy())

    print(trues)
    print(preds)

    accuracy, precision, recall, f1 = compute_metrics(trues, preds)

    return accuracy, precision, recall, f1


# PCT Models

In [24]:
sys.path.append('src/train')

from train_model import get_model

In [25]:

hidden_size = 64
k = 32

strat2name = {
    "1": "normal",
    "2": "tome_ft",
    "3": "tome_xyz",
    "4": "random"
}

for strat in strat2name:
    model, _ = get_model(
        "pct", 
        strat2name[strat], 
        1024,
        40,
        3,
        hidden_size,
        k,  
        "mps"
    )
    model.to("mps")

    fp = os.path.join("outputs/test", strat, "model.pt")
    state_dict = torch.load(fp)
    model.load_state_dict(state_dict)

    accuracy, precision, recall, f1 = compute_test_metrics(model, test_dl)
    print(accuracy, precision, recall, f1)

    


100%|██████████| 78/78 [04:45<00:00,  3.65s/it]


0.044165316045380876 0.042601391588757025 0.044165316045380876 0.04268158010120023


100%|██████████| 78/78 [04:22<00:00,  3.37s/it]


0.044975688816855756 0.03304237331766346 0.044975688816855756 0.03710854998276379


 50%|█████     | 39/78 [02:56<02:56,  4.52s/it]


KeyboardInterrupt: 

In [ ]:

hidden_size = 64
k = 32

model, _ = get_model(
    "pct", 
    "normal", 
    1024,
    40,
    3,
    hidden_size,
    k,  
    "mps"
)

print(model)

fp = os.path.join("outputs/test", "1", "model.pt")
state_dict = torch.load(fp)
model.load_state_dict(state_dict)

PCT(
  (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,), bias=False)
  (conv2): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (downsample1): FPS_KNN_PCT(
    (sample_and_group): Sample_Group()
    (local_op): Local_op(
      (conv1): Conv1d(128, 128, kernel_size=(1,), stride=(1,), bias=False)
      (conv2): Conv1d(128, 128, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
  )
  (downsample2): FPS_KNN_PCT(
    (sample_and_group): Sample_Group()
    (local_op): Local_op(
      (conv1): Conv1d(256, 256, kernel_size=(1,), stride=(1,), bias=False)
      (conv2): Conv1d(256, 256, kernel_size=(1

<All keys matched successfully>